In [ ]:
%matplotlib inline

import pandas as pd
import numpy as np
import rasterio as rs
import seaborn as sns
import os

from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils.vis_utils import plot_model
from keras.utils import plot_model
from matplotlib import pyplot as plt
from rasterio.plot import reshape_as_image
from rasterio.enums import Resampling
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import class_weight
from tqdm import tqdm

seed = 0
np.random.seed(seed)
palette = sns.color_palette("coolwarm", 5)
tqdm.pandas()

In [ ]:
model = load_model('../models/fallow_full.h5')

In [ ]:
plot_model(model, to_file='../models/fallow_full.png')

In [ ]:
df = pd.read_csv('../data/predict.csv')

In [ ]:
out_shape=(4, 50, 50)
resampling=Resampling.nearest 

data_path = '../data/sentinel/predict'

def calculate_band(row):
    raster = rs.open('%s/%s/%s' % (data_path, row['id'], row['file'])).read(
        out_shape=out_shape, resampling=resampling
    )
    image = reshape_as_image(raster)
    return image

def calculate_correctness(row):
    image = row['image']
    return np.count_nonzero(image) / image.size
    
df['image'] = df.progress_apply(calculate_band, axis=1)
df['correctness'] = df.progress_apply(calculate_correctness, axis=1)

df.head()

In [ ]:
sns.distplot(df.correctness, bins=20, kde=False, rug=True)

In [ ]:
df = df[df['correctness'] >= 0.9]

y_pred = model.predict(x=np.array(df.image.tolist()), verbose=1)

In [ ]:
def classify(row):
    if(row['activation'] > 0.5):
        return 1
    else:
        return 0

df['activation'] = y_pred
df['label'] = df.progress_apply(classify, axis=1)

result = df[['id', 'activation', 'label']]
result.to_csv('../data/result.csv')
result.head()